# Semantic Segmentation with on the fly Data Augmentation


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras_preprocessing.image import ImageDataGenerator
from gm_networks import unet_model

In [ ]:
# Load the initial input and output images and expand their dimensions to fit the ImageDataGenerator 
# For inputs: initial_dimensions = (width, height, 3) ---> new_dimensions = (1, width, height, 3)
# For outputs: initial_dimensions = (width, height) ---> new_dimensions = (1, width, height, 1)

x_initial_train = plt.imread('train/101-INPUT.jpg')
y_initial_train = plt.imread('train/101-OUTPUT-GT.png')
x_initial_train = np.expand_dims(x_initial_train, 0) 
y_initial_train = np.expand_dims(y_initial_train, (0, -1)) 

In [ ]:
# Define the new size to create images of size (NEW_SIZE, NEW_SIZE), as well as define the number of train and validation images to be created
NEW_SIZE = 512
TRAIN_IMAGES_NUMBER = 300

In [ ]:
# Concatenate the respective inputs and outputs to ensure that they will undergo the same crop
concatenated_images_train = np.concatenate([x_initial_train, y_initial_train], -1)

# Initialize the tensors for the training batches
croped_images_train = np.zeros([TRAIN_IMAGES_NUMBER, NEW_SIZE, NEW_SIZE, 4])

# Random croping
for i in range(TRAIN_IMAGES_NUMBER):
    
    t = tf.image.random_crop(concatenated_images_train, size=[1, NEW_SIZE, NEW_SIZE, 4])
    croped_images_train[i] = t[0]

# Separate the inputs from the outputs to create the final batches
x_train = croped_images_train[:, :, :, :3]
y_train = np.expand_dims(croped_images_train[:, :, :, 3], -1)

In [ ]:
# Create dictionaries with the alterations to be used for the data augmentation operations for inputs and outputs
""""
NOTE: It is important to use exact the same values for the parameters in both dictionaries. 
The reason why we create 2 dictionaries instead of a common one is because  we want to add the 
preprocessing function for the output masks (this function sets all the pixel values of the mask to 0 or 1. 
While the initial ground truth images are binary, after the data augmentation operations, such as rotations and 
shifts, pixels with intermediate values are appeared due to interpolation)
"""
x_datagen_args = dict(
    rotation_range=25, 
    width_shift_range=0.1, 
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect')

y_datagen_args = dict(
    rotation_range=25, 
    width_shift_range=0.1, 
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect', 
    preprocessing_function = lambda x: np.where(x>0, 1, 0).astype(x.dtype))

In [ ]:
BATCH_SIZE = 8

# Instatiate the generators
x_datagen = ImageDataGenerator(**x_datagen_args)
y_datagen = ImageDataGenerator(**y_datagen_args)

# Setting the same seed number for both generators leads to the same random alterations for both generators
seed = 19
x_datagen.fit(x_train, augment=True, seed=seed)
y_datagen.fit(y_train, augment=True, seed=seed)

x_generator_train = x_datagen.flow(x_train, seed=seed, batch_size=BATCH_SIZE)
y_generator_train = y_datagen.flow(y_train, seed=seed, batch_size=BATCH_SIZE)

train_generator = zip(x_generator_train, y_generator_train)

In [ ]:
INITIAL_FILTERS = 32
EPOCHS = 30
STEPS_PER_EPOCH = TRAIN_IMAGES_NUMBER // BATCH_SIZE

In [ ]:
from datetime import datetime
# Define the model and train
model = unet_model(input_shape=(NEW_SIZE, NEW_SIZE, 3), padding='same', initial_filters=INITIAL_FILTERS)
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

start = datetime.now()
history = model.fit(train_generator, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS)
stop = datetime.now()

minutes = (stop - start).total_seconds() / 60

print(minutes)

In [ ]:
import os
# save the model and history of the training (loss, accuracy)
model_path = f'models/Unet_fitgen_inputsize{NEW_SIZE}_initialfilters{INITIAL_FILTERS}_epochs{EPOCHS}_#TODO: loss and metric'
model.save(model_path)


In [ ]:
np.save(os.path.join(model_path, 'history'), history.history)